# Intro to Quantitative Textual Analysis - Week 10

## Diachronic corpora: Change over time (Brezina 2018, ch. 7)

### Key terms

- longitudinal study
- diachronic corpora
- lockwords
- bootstrapping test

### Visualization techniques

- Line graph
- Candlestick plot
- Sparkline

## Warm-up: N-grams

By now, we're all familiar with some charts created through Google's [Ngram Viewer](https://books.google.com/ngrams/). But what do these charts actually show us, and what are there limitations?

Search for a few terms of your choosing in the Ngram Viewer. Try changing the time scale or zooming in and out.

With a partner or in small groups, discuss the following questions:

1. What do these n-grams show us?
2. What corpus are they using?
3. What are the limitations of this kind of charting?
4. What data are we missing for more sophisticated analyses?

## Colors over time

In the ./data directory, you'll find 'colours-data.csv', a CSV dataset provided by Brezina. Each row has the year, followed by the relative frequencies of several colors for that year. We'll use these data to practice visualizing change over time.

In [ ]:
# install dependencies
%pip install altair pandas

In [ ]:
# import altair for visualization
import altair as alt

# import pandas for data-wrangling
import pandas as pd

In [ ]:
# load the data
colors_df = pd.read_csv("data/colours-data.csv")
colors_df

In [ ]:
# pivot the table for easier charting
value_vars = colors_df.columns.to_list()[1:]
by_color = pd.melt(
    colors_df, id_vars=["Year"], value_vars=value_vars, value_name="relative frequency"
).rename(columns={"variable": "color"})

by_color['Year'] = by_color['Year'].astype(str)

In [ ]:
# chart the data
alt.Chart(by_color).mark_line(point=alt.OverlayMarkDef(filled=False, fill="white")).encode(
    x="Year:T",
    y="relative frequency:Q",
    color=alt.Color("color").scale(None)
).properties(width=1000)

### Your turn

> Discuss: What kinds of information can you glean from this chart? How can you make the chart more useful?

1. Using the [Altair](https://altair-viz.github.io/user_guide/data.html) docs, add a tooltip showing the year, color, and relative frequency when you hover over a point.
2. The visualization helps very little for grey and blue -- can you figure out how to "zoom in" and get a meaningful sense of their change over time?
3. Can you regroup the data by decade (e.g., `[1600, 1609], [1610, 1619], etc.`) and plot the results as a [box plot](https://altair-viz.github.io/user_guide/marks/boxplot.html)?

# Bootstrapping and percentage change

In this section, we'll use the same dataset to explore "bootstrapping," a "process of multiple resampling" that "gives an insight into the amount of variation in the data and gives us the confidence to generalize from this sample" [@Brezina2018 231].

But first, let's start with a more intuitive method.

## Percentage change

```math
\text{\% increase/decrease} = \frac{\text{relative frequency in corpus 2} - \text{relative frequency in corpus 1}}{\text{relative frequency in corpus 1}} \times 100
```

Let's divide our colors corpus in two halves by year, corresponding to the two halves of the 17th Century.

In [ ]:
colors_df_1600_to_1649 = colors_df[colors_df['Year'].astype(int) <= 1649]
colors_df_1650_to_1699 = colors_df[colors_df['Year'].astype(int) >= 1650]

We'll use the mean relative frequency per color to calculate the percentage change.

In [ ]:
red_1600 = colors_df_1600_to_1649['red'].mean()
red_1650 = colors_df_1650_to_1699['red'].mean()
red_change = ((red_1650 - red_1600) / red_1600) * 100

red_change

### Your turn

Perform the same calculations for the other colors. 

Can you generalize the calculation of percentage change by writing a function?

## Bootstrap test

"The **bootstrap test** [proposed in [@Lijffijt.etal2016]] is a non-parametric test of statistical significance, ... which compares two corpora and computes the p-value associated with the comparison." [@Brezina2018 231–232]

We can define the bootstrapping test mathematically as follows:

```math
\text{p} = \frac{1 + 2 \times \text{number of bootstrapping cycles} \times \text{(p1 or 1 – p1, whichever is smaller)}}{1 + \text{number of bootstrapping cycles}}
```

where

```math
p1 = \frac{\text{For all boostrapping cycles sume of value H}}{\text{number of boostrapping cycles}}
```

and where H can be 1 (value of interest in corpus1 > corpus2), 0.5 (value of interest in corpus1 == corpus2), or 0 (value of interest corpus1 < corpus2).

**Note that resampling can include duplicates from the dataset.**

Again, you'll find the data that we're working with in the data/ folder -- this time, we're using the "bootstrap" CSVs (again provided by Brezina).

In [ ]:
df = pd.read_csv("./data/bootstrap_its.csv")

### Your turn

Calculate the p value for the bootstrap_*.csv data sets using Brezina's method. (Note that SciPy includes a `boostrap` method, but it differs from Brezina's definition.)

Be sure to calculate Cohen's _d_ and the 95% Confidence Interval as well. Are your results the same as Brezina's?